In [ ]:
# This code has been adapted and modified from IBM Qiskit 2021 and also from https://github.com/ttlion/ShorAlgQiskit.
# It uses the implementation as contained in the work of Stephane Beauregard (https://arxiv.org/abs/quant-ph/0205095)
# Many thanks to IBM Qiskit team, Tiago Miguel (ttlion), Qubit by Qubit, Peter Shor and Stephane Beauregard.

### Import libraries

In [ ]:
from typing import Optional, Union, Tuple, List
import math
import array
import fractions
import logging
import numpy as np
from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister, execute, IBMQ, transpile,BasicAer, Aer, assemble
from qiskit.circuit import Gate, Instruction, ParameterVector
from qiskit.circuit.library import QFT
from qiskit.providers import BaseBackend, Backend
from qiskit.quantum_info import partial_trace
from qiskit.utils import summarize_circuits
from qiskit.utils.arithmetic import is_power
from qiskit.utils.validation import validate_min
from qiskit.utils.quantum_instance import QuantumInstance
import qiskit.visualization

from qiskit.providers.aer import QasmSimulator
from datetime import datetime
import csv
# provider = IBMQ.enable_account("PUT TOKEN HERE")
backend = QasmSimulator()
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"  #"last_expr" or "all"

### Define functions

In [ ]:
def get_angles(a: int, n) -> np.ndarray:
        # """Calculates the array of angles to be used in the addition in Fourier Space."""
        s = bin(int(a))[2:].zfill(n + 1)
        angles = np.zeros([n + 1])
        for i in range(0, n + 1):
            for j in range(i, n + 1):
                if s[j] == '1':
                    angles[n - i] += math.pow(2, -(j - i))
            angles[n - i] *= np.pi
        return angles[::-1]
        # This returns the angles in the opposite order



def my_create_QFT(qft_num_qubits,approximation_degree: int = 0,do_swaps: bool = False,insert_barriers: bool = True, name: str = 'qft'):
#     up_reg = QuantumRegister(size = qft_num_qubits, name="aux")
    circuit_qft = QuantumCircuit(qft_num_qubits)
    i=qft_num_qubits-1
    while i>=0:
#         circuit_qft.h(up_reg[i])
        circuit_qft.h(i)
        j=i-1  
        while j>=0:
            if (np.pi)/(pow(2,(i-j))) > approximation_degree:
#                 circuit_qft.cu1( (np.pi)/(pow(2,(i-j))) , up_reg[i] , up_reg[j] )
                circuit_qft.cu1( (np.pi)/(pow(2,(i-j))) , i , j )
                j=j-1   
        if insert_barriers:
            circuit_qft.barrier()
        i=i-1  

    """ If specified, apply the Swaps at the end """
    if do_swaps:
        i=0
        while i < ((qft_num_qubits-1)/2):
#             circuit_qft.swap(up_reg[i], up_reg[qft_num_qubits-1-i])
            circuit_qft.swap(i, qft_num_qubits-1-i)
            i=i+1
    circuit_qft.name = "QFT"
    return circuit_qft

def my_create_inverse_QFT(qft_num_qubits,approximation_degree: int = 0,do_swaps: bool = False,insert_barriers: bool = True, name: str = 'iqft'):
    my_create_QFT_circuit = my_create_QFT(qft_num_qubits,approximation_degree,do_swaps,insert_barriers, name)
    my_create_inverse_QFT_circuit = my_create_QFT_circuit.inverse()
    my_create_inverse_QFT_circuit.name = "QFT†"
    return my_create_inverse_QFT_circuit


def phi_add_gate(size: int, angles: Union[np.ndarray, ParameterVector]) -> Gate:
#     """Gate that performs addition by a in Fourier Space."""
    circuit = QuantumCircuit(size, name="phi_add")
    for i, angle in enumerate(angles):
        circuit.p(angle, i)
    return circuit.to_gate()
    

def double_controlled_phi_add_mod_N(num_qubits: int, angles: Union[np.ndarray, ParameterVector],reg_size, a, N, n) -> QuantumCircuit:
    # """Creates a circuit which implements double-controlled modular addition by a."""
    circuit = QuantumCircuit(num_qubits, name="ccphi_add_mod_N")

    ctl_up = 0
    ctl_down = 1
    ctl_aux = 2

    # get qubits from aux register, omitting the control qubit
    qubits = range(3, num_qubits)

    # store the gates representing addition/subtraction by a in Fourier Space
    phi_add_a = phi_add_gate(len(qubits), angles)
    iphi_add_a = phi_add_a.inverse()
    
    phi_add_N = phi_add_gate(reg_size - 1, get_angles(N, n))
    iphi_add_N = phi_add_N.inverse()

    circuit.append(phi_add_a.control(2), [ctl_up, ctl_down, *qubits])
    circuit.append(iphi_add_N, qubits)
    
    qft = QFT(n + 1).to_instruction()
#     qft = my_create_QFT(n + 1).to_instruction()
    
    iqft = QFT(n + 1).inverse().to_instruction()
#     iqft = my_create_inverse_QFT(n + 1).to_instruction()
    circuit.append(iqft, qubits)

    circuit.cx(qubits[0], ctl_aux)
    
    circuit.append(qft, qubits)
    circuit.append(phi_add_N, qubits)
    circuit.append(iphi_add_a.control(2), [ctl_up, ctl_down, *qubits])
    circuit.append(iqft, qubits)

    circuit.x(qubits[0])
    circuit.cx(qubits[0], ctl_aux)
    circuit.x(qubits[0])

    circuit.append(qft, qubits)
    circuit.append(phi_add_a.control(2), [ctl_up, ctl_down, *qubits])
    return circuit

# """Circuit that implements single controlled modular multiplication by a"""
def controlled_multiple_mod_N(num_qubits: int, N: int, a: int, n, aux_reg_size):
#     """Implements modular multiplication by a as an instruction."""
    circuit = QuantumCircuit(
        num_qubits, 
#         name="multiply_by_{}_mod_{}".format(a % N, N),
        name=r"${0}^{{{1}^{{{2}}}}} mod{3}$".format(2,2,int(math.log(math.log(a,2),2)), N)
    )
#     label = r"${0}^{{{1}^{{{2}}}}} mod{3}$".format("†","y")
    down = circuit.qubits[1: n + 1]
    aux = circuit.qubits[n + 1:]
    qubits = [aux[i] for i in reversed(range(n + 1))]
    ctl_up = 0
    ctl_aux = aux[-1]
    
    angle_params = ParameterVector("angles", length=len(aux) - 1)
    double_controlled_phi_add = double_controlled_phi_add_mod_N(
        len(aux) + 2, angle_params, aux_reg_size, a, N, n
    )
    
    idouble_controlled_phi_add = double_controlled_phi_add.inverse()

    qft_circuit = QFT(n + 1).to_instruction()
#     qft_circuit = my_create_QFT(n + 1).to_instruction()

    iqft_circuit = QFT(n + 1).inverse().to_instruction()
#     iqft_circuit = my_create_inverse_QFT(n + 1).to_instruction()
    
    circuit.append(qft_circuit, qubits)
    
    # perform controlled addition by a on the aux register in Fourier space
    for i, ctl_down in enumerate(down):
        a_exp = (2 ** i) * a % N
        angles = get_angles(a_exp, n)
        bound = double_controlled_phi_add.assign_parameters({angle_params: angles})
        circuit.append(bound, [ctl_up, ctl_down, ctl_aux, *qubits])

    circuit.append(iqft_circuit, qubits)
    
    # perform controlled subtraction by a in Fourier space on both the aux and down register
    for j in range(n):
        circuit.cswap(ctl_up, down[j], aux[j])
    circuit.append(qft_circuit, qubits)
    
    a_inv = modinv(a, N)
    for i in reversed(range(len(down))):
        a_exp = (2 ** i) * a_inv % N
        angles = get_angles(a_exp, n)
        bound = idouble_controlled_phi_add.assign_parameters({angle_params: angles})
        circuit.append(bound, [ctl_up, down[i], ctl_aux, *qubits])

    circuit.append(iqft_circuit, qubits)
    
    return circuit

def modinv(a: int, m: int) -> int:
    # """Returns the modular multiplicative inverse of a with respect to the modulus m."""
    def egcd(a: int, b: int) -> Tuple[int, int, int]:
        if a == 0:
            return b, 0, 1
        else:
            g, y, x = egcd(b % a, a)
            return g, x - (b // a) * y, y

    g, x, _ = egcd(a, m)
    if g != 1:
        raise ValueError("The greatest common divisor of {} and {} is {}, so the "
                         "modular inverse does not exist.".format(a, m, g))
    return x % m

def get_factors(N: int, a: int, measurement: str) -> Optional[List[int]]:
    # """Apply the continued fractions to find r and the gcd to find the desired factors."""
    x_final = int(measurement, 2)
    #print('In decimal, x_final value for this result is: {}.'.format(x_final))

    if x_final <= 0:
        fail_reason = 'x_final value is <= 0, there are no continued fractions.'
    else:
        fail_reason = None
        #print('Running continued fractions for this case.')

    # Calculate T and x/T
    T_upper = len(measurement)
    T = pow(2, T_upper)
    x_over_T = x_final / T  ## this is our theta

    # Cycle in which each iteration corresponds to putting one more term in the
    # calculation of the Continued Fraction (CF) of x/T

    # Initialize the first values according to CF rule
    i = 0
    b = array.array('i')
    t = array.array('f')

    b.append(math.floor(x_over_T))
    t.append(x_over_T - b[i])

    exponential = 0.0
    while i < N and fail_reason is None:
        # From the 2nd iteration onwards, calculate the new terms of the CF based on the previous terms as the rule suggests
        if i > 0:
            try:
                b_temp = math.floor(1 / t[i - 1])
            except ZeroDivisionError as err:
                b_temp = 0
            b.append(b_temp)
            try:
                t_temp = (1 / t[i - 1]) - b[i]
            except ZeroDivisionError as err:
                t_temp = 0
            t.append(t_temp)  # type: ignore

        # Calculate the denominator of the CF using the known terms
        denominator = calculate_continued_fraction(b)

        # Increment i for next iteration
        i += 1

        if denominator % 2 == 1:
            #print('Odd denominator, will try next iteration of continued fractions.')
            continue

        # Denominator is even, try to get factors of N. Get the exponential a^(r/2)

        if denominator < 1000:
            try:
                exponential = pow(a, denominator / 2)
            except OverflowError as err:
                exponential = 999999999

        # Check if the value is too big or not
        if exponential > 1000000:
            if exponential == 999999999:
                fail_reason = 'OverflowError'
            else:
                fail_reason = 'denominator of continued fraction is too big (> 10^9).'
        else:
            # The value is not too big, get the right values and do the proper gcd()
            putting_plus = int(exponential + 1)
            putting_minus = int(exponential - 1)
            one_factor = math.gcd(putting_plus, N)
            other_factor = math.gcd(putting_minus, N)

            # Check if the factors found are trivial factors or are the desired factors
            if any(factor in {1, N} for factor in (one_factor, other_factor)):
                #print('Found just trivial factors, not good enough.')
                # Check if the number has already been found, (use i - 1 because i was already incremented)
                if t[i - 1] == 0:
                    fail_reason = 'the continued fractions found exactly x_final/(2^(2n)).'
            else:
                return sorted((one_factor, other_factor))

    # Search for factors failed, write the reason for failure to the debug logs
    #print('Cannot find factors from measurement {0} because {1}'.format(measurement, fail_reason or 'it took too many attempts.'))
    return None

def calculate_continued_fraction(b: array.array) -> int:
    # """Calculate the continued fraction of x/T from the current terms of expansion b."""

    x_over_T = 0

    for i in reversed(range(len(b) - 1)):
        x_over_T = 1 / (b[i + 1] + x_over_T)

    x_over_T += b[0]

    frac = fractions.Fraction(x_over_T).limit_denominator()

    #print('Approximation number %s of continued fractions:'.format(len(b)))
    #print("Numerator:{0} \t\t Denominator: {1}.".format(frac.numerator, frac.denominator))
    return frac.denominator

def process_results(sim_result, circuit, shots, N, a, n):
    counts_result = sim_result.get_counts(circuit)
    total_counts = len(counts_result)
    counts_result_sorted = sorted(counts_result.items(), key=lambda x: x[1], reverse=True)


    # """ Print info to user from the simulation results """
#     print('Printing the various results followed by how many times they happened (out of the {} cases):\n'.format(shots))
    
    counts_result_keys = list(counts_result.keys())
    counts_result_values = list(counts_result.values())
    
    #i=0
    #while i < len(counts_result):
        #print('Result \"{0}\" happened {1} times out of {2}\n'.format(list(sim_result.get_counts().keys())[i],list(sim_result.get_counts().values())[i],shots))
        #print('Result \"{0}\" happened {1} times out of {2}\n'.format(counts_result_keys[i],counts_result_values[i],shots))
        #i=i+1

    prob_success=0
    prob_failure=0
    result_successful_counts = 0
    result_failure_counts = 0
#     len(counts_result_sorted)
    
    # For each simulation result, print proper info to user and try to calculate the factors of N
    #for measurement in counts_result_keys:
    for measurement, frequency in counts_result_sorted:
        # Get the x_final value from the final state qubits
        x_value = int(measurement, 2)
        #prob_this_result = 100 * ( int(counts_result[measurement] ) ) / (shots)
        prob_this_result = 100 * frequency/shots
#         print("------> Analyzing result {0}. This result happened in {1:.4f} % of all cases\nIn decimal, x_final value for this result is: {2}".format(measurement,prob_this_result,x_value))
        factors = get_factors(N, a, measurement) 

        if factors:
            prob_success = prob_success + prob_this_result
#             print('Found factors {0} from measurement {1} which is {2} in decimal.\n'.format(factors, measurement, x_value))
            result_successful_counts = result_successful_counts + 1

            if factors not in result_factors:
                result_factors.append(factors)
        elif not factors:
            prob_failure = prob_failure + prob_this_result
            result_failure_counts = result_failure_counts + 1
            
        
    return [result_factors, prob_success, prob_failure, total_counts, result_successful_counts,result_failure_counts]

### Initialize variables

In [ ]:
def my_shor(a,N,shots):
    start_time_number = datetime.now()
    start_time = start_time_number.strftime("%H:%M:%S")
    summary_result = dict()
    validate_min('N', N, 3)
    validate_min('a', a, 2)


    if N < 1 or N % 2 == 0:
        raise ValueError('The input needs to be an odd integer greater than 1.')

    if a >= N or math.gcd(a, N) != 1:
        raise ValueError('The integer a needs to satisfy a < N and gcd(a, N) = 1.')

    n = math.ceil(math.log(N,2))
    global result_factors
    result_factors = []
    tf, b, p = is_power(N, return_decomposition=True)
    if tf:
        print('The input integer is a power: {0}={1}^{2}.'.format(N, b, p))
        result_factors.append(b)

    # """auxilliary quantum register used in addition and multiplication"""
    aux_reg = QuantumRegister(size = n+2, name="aux_reg")

    up_reg = QuantumRegister(2*n, name = "up_reg")

    # """quantum register where the multiplications are made"""
    down_reg = QuantumRegister(n, name = "down_reg")

    # """classical register where the measured values of the QFT are stored"""
    up_classic = ClassicalRegister(2*n, name="up_classic")

    # """ Create Quantum Circuit """
    circuit = QuantumCircuit(up_reg ,down_reg ,aux_reg, up_classic, name="Shor circuit(N={}, a={})".format(N, a))

    # phi_add_N_gate = phiADD(circuit,q,a,N,inv)
    phi_add_N_gate = phi_add_gate(aux_reg.size - 1, get_angles(N,n))
    iphi_add_N_gate = phi_add_N_gate.inverse()

    # """ Initialize down register to 1 and create maximal superposition in top register """
    circuit.h(up_reg)
    circuit.x(down_reg[0])
    # circuit.draw(filename = "shor_standard_QFT")

    # """ Apply the multiplication gates as showed in the report in order to create the exponentiation """
    for i, ctl_up in enumerate(up_reg):  # type: ignore
        a_aux = int(pow(a, pow(2, i)))
        controlled_multiple_mod_N_circuit = controlled_multiple_mod_N(
            len(down_reg) + len(aux_reg) + 1, N, a_aux,n,aux_reg.size
        )
        controlled_multiple_mod_N_result = controlled_multiple_mod_N_circuit.to_instruction()
        circuit.append(
            controlled_multiple_mod_N_result, [ctl_up, *down_reg, *aux_reg]
        )
#         circuit.draw()

    iqft = QFT(len(up_reg)).inverse().to_instruction()
    # iqft = my_create_inverse_QFT(len(up_reg)).to_instruction()
    # iqft = my_create_inverse_QFT(len(up_reg), insert_barriers = False).to_gate(label = r"$QFT^{{{0}}}$".format("†"))

    circuit.append(iqft, up_reg)
    circuit.measure(up_reg, up_classic)

#     circuit.draw(filename = "shor_standard_QFT_final_circuit",fold = -1 )

    #     print(summarize_circuits(circuit))
    # circuit.draw()

    print('Running with N={0} and a={1} with number of qubits n={2}'.format(N, a, 4*n + 2))
    
    qc_compiled = transpile(circuit, backend, optimization_level = 3)
    job_sim_1 = backend.run(qc_compiled, shots=shots)
    sim_result=job_sim_1.result()

#     counts_result = sim_result.get_counts(circuit)
#     len(counts_result)
#     measurement_plot = qiskit.visualization.plot_histogram(counts_result,figsize=(20, 12) ,number_to_keep = 30,bar_labels=True, title = "Measurement results from shor_standard_QFT circuit variant" )

    # measurement_plot.savefig("shor_standard_QFT_measurement")
#     measurement_plot

    processed_result = process_results(sim_result, circuit, shots, N, a, n)
    end_time_number = datetime.now()
    end_time = end_time_number.strftime("%H:%M:%S")
    duration = end_time_number - start_time_number


    print("Current Start Time =", start_time)
    print(processed_result)
    print("Current End Time =", end_time)

    circuit_count_ops = circuit.count_ops()
    circuit_decomposed = circuit.decompose()
    circuit_decomposed_count_ops = circuit_decomposed.count_ops()
    qc_compiled_count_ops = qc_compiled.count_ops()

    summary_result["num_qubits"] = n
    summary_result["Number(N)"] = N
    summary_result["a"] = a
    summary_result["start_time"] = start_time
    summary_result["end_time"] = end_time
    summary_result["duration"] = duration
    summary_result["result_factors"] = processed_result[0]
    summary_result["prob_success"] = processed_result[1]
    summary_result["prob_failure"] = processed_result[2]
    summary_result["total_counts"] = processed_result[3]
    summary_result["result_successful_counts"] = processed_result[4]
    summary_result["result_failure_counts"] = processed_result[5]


    summary_result["circuit_width"] = circuit.width()
    summary_result["circuit_depth"] = circuit.depth()
    summary_result["circuit_size"] = circuit.size() 
    summary_result["circuit_num_nonlocal_gates"] = circuit.num_nonlocal_gates()
    summary_result["circuit_num_ancillas"] = circuit.num_ancillas
    summary_result["circuit_num_clbits"] = circuit.num_clbits
    summary_result["circuit_num_qubits"] = circuit.num_qubits
    summary_result["circuit_num_ancillas"] = circuit.num_ancillas
    summary_result["circuit_num_of_count_ops"] = len(circuit_count_ops)
    summary_result["circuit_num_of_x"] = circuit_count_ops.get('x')
    summary_result["circuit_num_of_measure"] = circuit_count_ops.get('measure')
    summary_result["circuit_num_of_h"] = circuit_count_ops.get('h')
    summary_result["circuit_num_of_cswap"] = circuit_count_ops.get('cswap')
    summary_result["circuit_num_of_swap"] = circuit_count_ops.get('swap')
    summary_result["circuit_num_of_cx"] = circuit_count_ops.get('cx')
    summary_result["circuit_num_of_toffoli"] = circuit_count_ops.get('toffoli')
    summary_result["circuit_num_of_p"] = circuit_count_ops.get('p')
    summary_result["circuit_num_of_t"] = circuit_count_ops.get('t')

    summary_result["circuit_decomposed_width"] = circuit_decomposed.width()
    summary_result["circuit_decomposed_depth"] = circuit_decomposed.depth()
    summary_result["circuit_decomposed_size"] = circuit_decomposed.size() 
    summary_result["circuit_decomposed_num_nonlocal_gates"] = circuit_decomposed.num_nonlocal_gates()
    summary_result["circuit_decomposed_num_ancillas"] = circuit_decomposed.num_ancillas
    summary_result["circuit_decomposed_num_clbits"] = circuit_decomposed.num_clbits
    summary_result["circuit_decomposed_num_qubits"] = circuit_decomposed.num_qubits
    summary_result["circuit_decomposed_num_ancillas"] = circuit_decomposed.num_ancillas
    summary_result["circuit_decomposed_num_of_count_ops"] = len(circuit_decomposed_count_ops)
    summary_result["circuit_decomposed_num_of_x"] = circuit_decomposed_count_ops.get('x')
    summary_result["circuit_decomposed_num_of_measure"] = circuit_decomposed_count_ops.get('measure')
    summary_result["circuit_decomposed_num_of_h"] = circuit_decomposed_count_ops.get('h')
    summary_result["circuit_decomposed_num_of_cswap"] = circuit_decomposed_count_ops.get('cswap')
    summary_result["circuit_decomposed_num_of_swap"] = circuit_decomposed_count_ops.get('swap')
    summary_result["circuit_decomposed_num_of_cx"] = circuit_decomposed_count_ops.get('cx')
    summary_result["circuit_decomposed_num_of_toffoli"] = circuit_decomposed_count_ops.get('toffoli')
    summary_result["circuit_decomposed_num_of_p"] = circuit_decomposed_count_ops.get('p')
    summary_result["circuit_decomposed_num_of_t"] = circuit_decomposed_count_ops.get('t')

    summary_result["qc_compiled_width"] = qc_compiled.width()
    summary_result["qc_compiled_depth"] = qc_compiled.depth()
    summary_result["qc_compiled_size"] = qc_compiled.size() 
    summary_result["qc_compiled_num_nonlocal_gates"] = qc_compiled.num_nonlocal_gates()
    summary_result["qc_compiled_num_ancillas"] = qc_compiled.num_ancillas
    summary_result["qc_compiled_num_clbits"] = qc_compiled.num_clbits
    summary_result["qc_compiled_num_qubits"] = qc_compiled.num_qubits
    summary_result["qc_compiled_num_ancillas"] = qc_compiled.num_ancillas
    summary_result["qc_compiled_num_of_count_ops"] = len(qc_compiled_count_ops)
    summary_result["qc_compiled_num_of_x"] = qc_compiled_count_ops.get('x')
    summary_result["qc_compiled_num_of_measure"] = qc_compiled_count_ops.get('measure')
    summary_result["qc_compiled_num_of_h"] = qc_compiled_count_ops.get('h')
    summary_result["qc_compiled_num_of_cswap"] = qc_compiled_count_ops.get('cswap')
    summary_result["qc_compiled_num_of_swap"] = qc_compiled_count_ops.get('swap')
    summary_result["qc_compiled_num_of_cx"] = qc_compiled_count_ops.get('cx')
    summary_result["qc_compiled_num_of_toffoli"] = qc_compiled_count_ops.get('toffoli')
    summary_result["qc_compiled_num_of_p"] = qc_compiled_count_ops.get('p')
    summary_result["qc_compiled_num_of_t"] = qc_compiled_count_ops.get('t')

    return summary_result

### Process the results

In [ ]:
# Run for just a single number N
%%time
N = 33
shots = 1024
global result_factors

all_summary_result_temp = []
for random_a in range(16, 17):
    if math.gcd(random_a,N) > 1:
        continue
    a = random_a
    summary_result = my_shor(a,N,shots)
    print("Finished running for a = {} and N = {}\n".format(a, N))
    all_summary_result_temp.append(summary_result)
    summary_result_list = []
    for key, value in summary_result.items():
        summary_result_list.append([key,value])
    summary_result_list
    
    with open("a({0})_N({1})_standard.csv".format(a, N), 'a') as myfile:
        write = csv.writer(myfile)
        #write.writerow(fields)
        write.writerows(summary_result_list)

all_summary_result_temp

In [ ]:
# Run for many numbers N.
%%time
shots = 1024
global result_factors
all_summary_result = []
for N in [15, 21, 33, 35, 39, 51, 55, 57]:
    for a in range(2, N):
        if math.gcd(a,N) > 1:
            continue
        print("Beginning running for a = {} and N = {}".format(a, N))
        summary_result = my_shor(a,N,shots)
        print("Finished running for a = {} and N = {}\n\n".format(a, N))
        all_summary_result.append(summary_result)

all_summary_result

In [ ]:
%qiskit_copyright